In [ ]:
#submit1. score:0.20629 crossvalidation: 0.20592940285767342
#submit2. score:0.18104 crossvalidation: 0.1822781167369381
#submit3. score:0.16517 crossvalidation: 0.16455070183341483
#submit4. score:0.18170 crossvalidation: 0.18388329418394642

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (22,8)

import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error

**Чистим от столбцов, в которых есть пропуски в данных**

In [ ]:
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']
train_clear = train.drop(cols_to_drop, axis=1)


**Находим столбцы, в которых малоинформативные признаки и избавляемся от них**

In [ ]:
num_rows = len(train_clear.index)
low_information_cols = [] #

for col in train_clear.columns:
    cnts = train_clear[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct > 0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

In [ ]:
train_fin = train_clear.drop(low_information_cols, axis=1)

**Находим столбцы с числовыми значениями.** С категориальными особо сегодня не хочу работать :(

In [ ]:
train_numeric = train_fin.select_dtypes(include=[np.number])
numeric_cols = train_numeric.columns.values
print(numeric_cols)

In [ ]:
pd.options.display.max_columns = 200
train_numeric.head()

In [ ]:
plt.figure(figsize = (8,8))
plt.hist(train['TotalBsmtSF'], bins = 100, label = 'train', alpha = 0.5)
plt.hist(test['TotalBsmtSF'], bins = 100, label = 'test', alpha = 0.5)
plt.show()

***Делаем логарифмы величин***

In [ ]:
train['LotAreaLog'] = np.log1p(train['LotArea'])
test['LotAreaLog'] = np.log1p(test['LotArea'])
train['SalePriceLog'] = np.log1p(train['SalePrice'])
# GrLivArea
train['GrLivAreaLog'] = np.log1p(train['GrLivArea'])
test['GrLivAreaLog'] = np.log1p(test['GrLivArea'])
# 1stFlrSF
train['1stFlrSFLog'] = np.log1p(train['1stFlrSF'])
test['1stFlrSFLog'] = np.log1p(test['1stFlrSF'])
# TotRmsAbvGrd показывает странное поведение при логарифмировании
train['TotRmsAbvGrdLog'] = np.log1p(train['TotRmsAbvGrd'])
test['TotRmsAbvGrdLog'] = np.log1p(test['TotRmsAbvGrd'])


In [ ]:
train['GrLivAreaLog'].corr(train['1stFlrSFLog'])

In [ ]:
plt.figure(figsize = (8,8))
plt.hist(train['LotAreaLog'], bins = 100, label = 'train', alpha = 0.5)
plt.hist(test['LotAreaLog'], bins = 100, label = 'test', alpha = 0.5)
plt.show()

In [ ]:
plt.figure(figsize = (8,8))
plt.hist(train['SalePriceLog'], bins = 100, label = 'train', alpha = 0.5)
plt.show()

In [ ]:
plt.figure(figsize= (8,8))
plt.scatter( train['TotalBsmtSF'], train['SalePriceLog'])
plt.xlabel('TotalBsmtSF')
plt.ylabel('SalePriceLog')
plt.show()

# **Кроссвалидация тут**

In [ ]:
kf = KFold(n_splits = 5)
kf.get_n_splits(train)

In [ ]:
# features1 = ['LotAreaLog', 'OverallQual', 'GrLivAreaLog', '1stFlrSFLog', 'YearBuiltNorm', 'GarageAreaNorm']
features1 = ['LotAreaLog', 'OverallQual', 'GrLivAreaLog', '1stFlrSFLog', 'YearBuiltNorm', 'TotRmsAbvGrd']
features2 = ['TotalBsmtSF' ]

In [ ]:
for f in features1:
    plt.figure(figsize = (8,8))
    plt.hist(train[f], bins = 100, label = 'train', alpha = 0.5)
    plt.hist(test[f], bins = 100, label = 'test', alpha = 0.5)
    plt.show()

In [ ]:
for f in features1:
    plt.figure(figsize= (8,8))
    plt.scatter(train[f], train['SalePriceLog'])
    plt.xlabel(f)
    plt.ylabel('SalePriceLog')
    plt.show()

In [ ]:
_min = min(train['LotAreaLog'])
_max = max(train['LotAreaLog'])
train['LotAreaLogNorm'] = (train['LotAreaLog'] - _min)/(_max - _min)
_min = min(train['OverallQual'])
_max = max(train['OverallQual'])
train['OverallQualNorm'] = (train['OverallQual'] - _min)/(_max - _min)
_min = min(train['GrLivAreaLog'])
_max = max(train['GrLivAreaLog'])
train['GrLivAreaLogNorm'] = (train['GrLivAreaLog'] - _min)/(_max - _min)
_min = min(train['1stFlrSFLog'])
_max = max(train['1stFlrSFLog'])
train['1stFlrSFLogNorm'] = (train['1stFlrSFLog'] - _min)/(_max - _min)
_min = min(train['TotRmsAbvGrd'])
_max = max(train['TotRmsAbvGrd'])
train['TotRmsAbvGrdNorm'] = (train['TotRmsAbvGrd'] - _min)/(_max - _min)

features1norm = ['LotAreaLogNorm', 'OverallQualNorm', 'GrLivAreaLogNorm', '1stFlrSFLogNorm', 'TotRmsAbvGrdNorm']

In [ ]:
# YearBuilt
_min = min(train['YearBuilt'])
_max = max(train['YearBuilt'])
train['YearBuiltNorm'] = (train['YearBuilt'] - _min)/(_max - _min)
_min = min(test['YearBuilt'])
_max = max(test['YearBuilt'])
test['YearBuiltNorm'] = (test['YearBuilt'] - _min)/(_max - _min)

In [ ]:
# GarageArea
_min = min(train['GarageArea'])
_max = max(train['GarageArea'])
train['GarageAreaNorm'] = (train['GarageArea'] - _min)/(_max - _min)
_min = min(test['GarageArea'])
_max = max(test['GarageArea'])
test['GarageAreaNorm'] = (test['GarageArea'] - _min)/(_max - _min)

In [ ]:
print(kf)
reg_model = LinearRegression()
rmse_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, features1 ], train.loc[test_index, features1]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'], train.loc[test_index, 'SalePriceLog']

    reg_model.fit(X_train, y_train)
    predict = reg_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(predict,y_test))
    rmse_list.append(rmse)
    print(i, rmse)
    
print("Mean rmse for 5-fold: {}".format(str(np.mean(rmse_list))))
print("std rmse for 5-fold: {}".format(str(np.std(rmse_list))))

1 аргумент:
* Mean rmse for 5-fold: 0.36610553458247586
* std rmse for 5-fold: 0.014223113160126786

2 аргумента:
* Mean rmse for 5-fold: 0.20592940285767342
* std rmse for 5-fold: 0.009261118671450628

![](http://)3 аргумента: 
* Mean rmse for 5-fold: 0.18819389122943653
* std rmse for 5-fold: 0.008821647379472804

4 аргумента:
* Mean rmse for 5-fold: 0.1822781167369381
* std rmse for 5-fold: 0.009271730315584328

5 аргументов:
* Mean rmse for 5-fold: 0.16455070183341483
* std rmse for 5-fold: 0.011576261138424908

# SGDRegressor

In [ ]:
print(kf)
sgd_model = SGDRegressor(max_iter = 1000000)
rmse_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, features1 ], train.loc[test_index, features1]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'], train.loc[test_index, 'SalePriceLog']

    sgd_model.fit(X_train, y_train)
    predict = sgd_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(predict,y_test))
    rmse_list.append(rmse)
    print(i, rmse, 'coef', sgd_model.coef_)
    
print("Mean rmse for 5-fold: {}".format(str(np.mean(rmse_list))))
print("std rmse for 5-fold: {}".format(str(np.std(rmse_list))))

# SGD+Linear

In [ ]:
print(kf)
reg_model = LinearRegression()
sgd_model = SGDRegressor(max_iter = 1000000)
rmse_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, features1], train.loc[test_index, features1]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'], train.loc[test_index, 'SalePriceLog']

    sgd_model.fit(X_train, y_train)
    reg_model.fit(X_train, y_train)
    predict = (sgd_model.predict(X_test) + reg_model.predict(X_test))/2
    rmse = np.sqrt(mean_squared_error(predict,y_test))
    rmse_list.append(rmse)
    print(i, rmse, 'coef', sgd_model.coef_)
    
print("Mean rmse for 5-fold: {}".format(str(np.mean(rmse_list))))
print("std rmse for 5-fold: {}".format(str(np.std(rmse_list))))

# **Обучение модели**

In [ ]:
reg_model.coef_

In [ ]:
reg_model = LinearRegression()
reg_model.fit(train[features1], train['SalePriceLog'])
reg_model.coef_

sgd_model = SGDRegressor(max_iter = 1000000)
sgd_model.fit(train[features1], train['SalePriceLog'])
sgd_model.coef_

In [ ]:
prediction = (reg_model.predict(test[features1]) + sgd_model.predict(test[features1]))/2

sample_submission = pd.read_csv ("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
sample_submission['SalePrice'] = np.exp(prediction)

In [ ]:
sample_submission.head

In [ ]:
sample_submission.to_csv('submission5.csv', index = None)